In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [3]:
company_name = "Apple"
job_title = "Software engineer"
location = "India"
# add headless option
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(options = options)
# url = f"https://www.glassdoor.co.in/Salary/Google-Software-Engineer-India-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"
# https://www.glassdoor.co.in/Salary/Apple-Software-Engineer-Salaries-E1138_D_KO6,23.htm
url = f"https://www.glassdoor.co.in/Salary/{company_name}-{job_title.replace(' ', '-')}-{location}-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"

driver.get(url)
driver.maximize_window()
# wait for the page to load completely
time.sleep(2)
# collect raw html
response = driver.page_source
# driver.close()

## Now to parse and scan the html tree

In [2]:
import re
from html.parser import HTMLParser

class customizedParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.result = []
    def handle_data(self, data):
        self.result.append(data)
        
def do():
    company_name = "Google"
    job_title = "Software engineer"
    location = "India"
    # add headless option
    options = Options()
    # options.add_argument("--headless")
    driver = webdriver.Chrome(options = options)
    # url = f"https://www.glassdoor.co.in/Salary/Google-Software-Engineer-India-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"
    url = f"https://www.glassdoor.co.in/Salary/{company_name}-{job_title.replace(' ', '-')}-{location}-Salaries-EJI_IE9079.0,6_KO7,24_IL.25,30_IN115.htm"
    
    driver.get(url)
    driver.maximize_window()
    # wait for the page to load completely
    time.sleep(2)
    # collect raw html
    response = driver.page_source
    # driver.close()
    soup = BeautifulSoup(response, 'html.parser')
    driver.close()
    # Locate the section
    section = soup.find("section", class_ = "hero_SubModule__wCYtE")
    # if the section is not present
    if not section:
        return {
            "error" : "No salary range defined"
        }
    div = section.find('div', class_ = 'hero_TotalPayLayout__X55hl hero_PayRange__nKzVj')
    div_salary_range = div.find_all('div', 'hero_PayRange__nKzVj')
    # print(div_salary_range) # we got a list like
    '''
[<div class="hero_PayRange__nKzVj">₹15.0L</div>, <div class="hero_PayRange__nKzVj">-</div>, 
<div class="hero_PayRange__nKzVj">₹35.0L<sub class="hero_PayLabel__UtI9Y">/yr</sub></div>]
    '''
    salary = ""
    for unparsed_data in div_salary_range:
         # instantiating our customized HTM parser
        parser = customizedParser()
        # feed the unparsed data to parser
        parser.feed(repr(unparsed_data))
        # get the resultant list
        salary += ' '.join(parser.result)
    return salary

In [3]:
print(do())

₹15.0L-₹35.0L /yr


In [45]:
from html.parser import HTMLParser

class TextExtractor(HTMLParser):
    def __init__(self):
        super().__init__()
        self.result = []

    def handle_data(self, data):
        self.result.append(data.strip())

html = '<div class="hero_PayRange__nKzVj">₹35.0L<sub class="hero_PayLabel__UtI9Y">/yr</sub></div>'

parser = TextExtractor()
parser.feed(html)
output = ''.join(parser.result)
print(output)

₹35.0L/yr


## **HTML Parser library**
```html.parser -> feed()```

In [47]:
from html.parser import HTMLParser
# We need to customize the parser or else it contains blank placeholders
class customizedParser(HTMLParser):
    def __init__(self):
        # Call parent class constructor
        super().__init__()
        # Container to hold data
        self.result = []
    # customized the way to handle data
    def handle_data(self, data):
        self.result.append(data.strip())

# __main__ segment
' instantiate customized html parser '
myParser = customizedParser()
unparsed = '<div class="hero_PayRange__nKzVj">₹35.0L<sub class="hero_PayLabel__UtI9Y">/yr</sub></div>'
# use feed function
myParser.feed(unparsed)
parsed = myParser.result
print(parsed)

['₹35.0L', '/yr']


# **Using levels.fyi**
Let's try to open levels.fyi
Sample link - ```https://www.levels.fyi/companies/google/salaries/software-engineer?country=254```

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time

In [44]:
def do(company, job):
    options = Options()
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options = options)
    url = f"https://www.levels.fyi/companies/{company}/salaries/{job.replace(' ','-')}"
    driver.get(url)
    driver.maximize_window()
    time.sleep(2)
    # get the raw html
    response = driver.page_source
    soup = BeautifulSoup(response, 'html.parser')
    driver.close()
    # get the table
    table = soup.find('table', class_ = 'MuiTable-root css-1f6fkxk')
    if not table:
        return None
    print('table found')
    result = []
    # find tbody
    tbody = table.find('tbody', class_ = 'MuiTableBody-root job-family_tableBody__MaiIw css-1xnox0e')
    if not tbody:
        return None
    print('found tbody')
    # find all rows
    rows = tbody.find_all('tr', class_ = 'MuiTableRow-root job-family_bodyRow__WuAug css-140sacz')
    # traver each row
    for row in rows:
        # get columns as td value
        col = row.find_all('td', class_ = 'MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-sizeMedium css-k1mugk')
        # we want minimum results
        if len(col) >= 5:
            salary_record = {
                'level': col[0].get_text(strip = True),
                'Total': col[1].get_text(strip = True),
                'Base': col[2].get_text(strip = True),
                'Stock (/yr)': col[3].get_text(strip = True),
                'Bonus': col[4].get_text(strip = True)
            }
            result.append(salary_record)
    return result

In [55]:
company = input('Enter company name : ')
job = input('Enter job : ')
print(do(company, job))

Enter company name :  microsoft
Enter job :  software engineer


table found
found tbody
[{'level': 'SDE59(Entry Level)', 'Total': '₹13.87M', 'Base': '₹10.67M', 'Stock (/yr)': '₹2.43M', 'Bonus': '₹776K'}, {'level': '60', 'Total': '₹14.56M', 'Base': '₹11.28M', 'Stock (/yr)': '₹2.53M', 'Bonus': '₹753K'}, {'level': 'SDE II61', 'Total': '₹16.56M', 'Base': '₹12.65M', 'Stock (/yr)': '₹2.51M', 'Bonus': '₹1.4M'}, {'level': '62', 'Total': '₹17.83M', 'Base': '₹14.27M', 'Stock (/yr)': '₹2.35M', 'Bonus': '₹1.2M'}]


### **We will also be expanding results using xpath**

In [9]:
def do(company, job):
    options = Options()
    options.add_argument("--disable-gpu")
    options.add_argument("--headless")
    driver = webdriver.Chrome(options = options)
    url = f"https://www.levels.fyi/companies/{company}/salaries/{job.replace(' ','-')}"
    driver.get(url)
    driver.maximize_window()
    
    try:
        # Wait for the clickable button
        wait = WebDriverWait(driver, 10)
        view_more_levels = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textNeutral MuiButton-sizeLarge MuiButton-textSizeLarge MuiButton-colorNeutral MuiButton-root MuiButton-text MuiButton-textNeutral MuiButton-sizeLarge MuiButton-textSizeLarge MuiButton-colorNeutral css-y5b368']")))
        view_more_levels.click()
        print('clicked sir')
        time.sleep(2)
    except Exception as e:
        print('button not found:', e)
    # get the raw html
    response = driver.page_source
    soup = BeautifulSoup(response, 'html.parser')
    driver.close()
    # get the table
    table = soup.find('table', class_ = 'MuiTable-root css-1f6fkxk')
    if not table:
        return None
    print('table found')
    result = []
    # find tbody
    tbody = table.find('tbody', class_ = 'MuiTableBody-root job-family_tableBody__MaiIw css-1xnox0e')
    if not tbody:
        return None
    print('found tbody')
    # find all rows
    rows = tbody.find_all('tr', class_ = 'MuiTableRow-root job-family_bodyRow__WuAug css-140sacz')
    # traver each row
    for row in rows:
        # get columns as td value
        col = row.find_all('td', class_ = 'MuiTableCell-root MuiTableCell-body MuiTableCell-alignLeft MuiTableCell-sizeMedium css-k1mugk')
        # we want minimum results
        if len(col) >= 5:
            salary_record = {
                'level': col[0].get_text(strip = True),
                'Total': col[1].get_text(strip = True),
                'Base': col[2].get_text(strip = True),
                'Stock (/yr)': col[3].get_text(strip = True),
                'Bonus': col[4].get_text(strip = True)
            }
            result.append(salary_record)
    return result

In [10]:
print(do('microsoft', 'software engineer'))

clicked sir
table found
found tbody
[{'level': 'SDE59(Entry Level)', 'Total': '₹13.87M', 'Base': '₹10.67M', 'Stock (/yr)': '₹2.43M', 'Bonus': '₹776K'}, {'level': '60', 'Total': '₹14.56M', 'Base': '₹11.28M', 'Stock (/yr)': '₹2.53M', 'Bonus': '₹753K'}, {'level': 'SDE II61', 'Total': '₹16.56M', 'Base': '₹12.65M', 'Stock (/yr)': '₹2.51M', 'Bonus': '₹1.4M'}, {'level': '62', 'Total': '₹17.83M', 'Base': '₹14.27M', 'Stock (/yr)': '₹2.35M', 'Bonus': '₹1.2M'}, {'level': 'Senior SDE63', 'Total': '₹19.99M', 'Base': '₹15.42M', 'Stock (/yr)': '₹2.95M', 'Bonus': '₹1.61M'}, {'level': '64', 'Total': '₹23.86M', 'Base': '₹16.73M', 'Stock (/yr)': '₹4.75M', 'Bonus': '₹2.38M'}, {'level': 'Principal SDE65', 'Total': '₹27.39M', 'Base': '₹18.18M', 'Stock (/yr)': '₹6.42M', 'Bonus': '₹2.79M'}, {'level': '66', 'Total': '₹32.91M', 'Base': '₹19.19M', 'Stock (/yr)': '₹10.67M', 'Bonus': '₹3.05M'}, {'level': '67', 'Total': '₹49.26M', 'Base': '₹21.83M', 'Stock (/yr)': '₹20.61M', 'Bonus': '₹6.81M'}, {'level': 'Partner68